In [1]:
import warnings
warnings.filterwarnings("always")

In [3]:
import numpy as np, matplotlib.pyplot as plt
import keras

In [4]:
'''
LOAD PRE-TRAINED MODEL
'''

'\nLOAD PRE-TRAINED MODEL\n'

In [5]:
from keras.applications import VGG16

#Load the VGG model
vgg16 = VGG16(include_top=False, input_shape=(224, 224, 3)) # load vgg16 with all defaults except top

In [6]:
'''
MARK ALL LAYERS AS NON-TRAINABLE SO TRAINING DOESN'T ALTER WEIGHTS
'''

"\nMARK ALL LAYERS AS NON-TRAINABLE SO TRAINING DOESN'T ALTER WEIGHTS\n"

In [7]:
for layer in vgg16.layers:
    layer.trainable = False

In [8]:
print vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [9]:
'''
TUNE MODEL TO AUTOFOCUS NEEDS
'''

'\nTUNE MODEL TO AUTOFOCUS NEEDS\n'

In [10]:
from keras import layers

# my_model_output = vgg16.layers[10].output # Only use the first three blocks of convolution
my_model_output = vgg16.output # use all 5 conv layers of vgg
my_model_output = layers.GlobalAveragePooling2D()(my_model_output) # Then add a GlobalAveragePooling layer to get 1d representation
# my_model_output = layers.Dense(128, activation='relu')(my_model_output) # add a Dense layer just to increase trainable params
# my_model_output = layers.Dropout(0.5)(my_model_output) # Add regularization to help decrease overfitting data
my_model_output = layers.Dense(2, activation='softmax')(my_model_output) # binary classification prediction layer

my_model = keras.models.Model(inputs=vgg16.input, outputs=my_model_output) #created my neural network

my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [11]:
'''
SET UP TRAINING AND VALIDATION DATA
'''

'\nSET UP TRAINING AND VALIDATION DATA\n'

In [10]:
from keras.preprocessing.image import ImageDataGenerator, load_img

train_datagen = ImageDataGenerator(
                            rescale=1./255,
                            rotation_range=20,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            horizontal_flip=True,
                            vertical_flip=True,
                            fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_batchsize = 20
val_batchsize = 5

train_generator = train_datagen.flow_from_directory(
                                                'train_data',
                                                target_size=(224, 224),
                                                batch_size=train_batchsize,
                                                class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
                                                    'validation_data',
                                                    target_size=(224, 224),
                                                    batch_size=val_batchsize,
                                                    class_mode='categorical',
                                                    shuffle=False)

Found 1400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [13]:
'''
COMPILE AND TRAIN MODEL
'''

'\nCOMPILE AND TRAIN MODEL\n'

In [14]:
# Compile
try:
    my_model = keras.models.load_model('autofocus_vgg16.h5') # trains from last epoch if model exists
except:
    print "Training new model..."
    my_model.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['acc'])

# Train
checkpoint = keras.callbacks.ModelCheckpoint('autofocus_vgg16.h5', monitor='val_loss', verbose=1, period=1)

my_model_hist = my_model.fit_generator(
                            train_generator,
                            steps_per_epoch=2*train_generator.samples/train_generator.batch_size ,
                            epochs=25,
                            validation_data=validation_generator,
                            validation_steps=validation_generator.samples/validation_generator.batch_size,
                            verbose=1,
                            callbacks=[checkpoint])

# Save the Model
my_model.save('autofocus_vgg16.h5')

Training new model...
Epoch 1/25
140/140 [==============================] - 1348s 10s/step - loss: 0.6583 - acc: 0.6286 - val_loss: 0.6194 - val_acc: 0.7017

Epoch 00001: saving model to autofocus_vgg16.h5
Epoch 2/25
140/140 [==============================] - 1346s 10s/step - loss: 0.6095 - acc: 0.6796 - val_loss: 0.5831 - val_acc: 0.7200

Epoch 00002: saving model to autofocus_vgg16.h5
Epoch 3/25
140/140 [==============================] - 1353s 10s/step - loss: 0.5816 - acc: 0.7075 - val_loss: 0.5773 - val_acc: 0.7117

Epoch 00003: saving model to autofocus_vgg16.h5
Epoch 4/25
140/140 [==============================] - 1350s 10s/step - loss: 0.5634 - acc: 0.7214 - val_loss: 0.5445 - val_acc: 0.7367

Epoch 00004: saving model to autofocus_vgg16.h5
Epoch 5/25
140/140 [==============================] - 1349s 10s/step - loss: 0.5577 - acc: 0.7300 - val_loss: 0.5307 - val_acc: 0.7450

Epoch 00005: saving model to autofocus_vgg16.h5
Epoch 6/25
140/140 [==============================] - 1348

In [11]:
'''
PREDICT
'''

'\nPREDICT\n'

In [12]:
import numpy as np
import keras
from keras.models import load_model
from keras.preprocessing import image

In [13]:
model = keras.models.load_model('autofocus_vgg16.h5')

In [14]:
def load_image_as_tensor(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.
    return img

def predict(img_path):
    img = load_image_as_tensor(img_path)
    prob = model.predict(img)
    classes = np.argmax(prob, axis=1)
    label_map = (validation_generator.class_indices)
    print img_path
    print prob, classes, label_map

In [15]:
predict('test_blur/black_square.jpg') 
predict('test_blur/black_square9.jpg')
predict('test_blur/black_square17.jpg')
predict('test_blur/black_square25.jpg')
predict('test_blur/black_square33.jpg')
predict('test_blur/black_square41.jpg')

# 100% Accurate for these 6 test images

test_blur/black_square.jpg
[[0.54110146 0.4588986 ]] [0] {'focused': 0, 'unfocused': 1}
test_blur/black_square9.jpg
[[0.46069276 0.53930724]] [1] {'focused': 0, 'unfocused': 1}
test_blur/black_square17.jpg
[[0.35018128 0.6498188 ]] [1] {'focused': 0, 'unfocused': 1}
test_blur/black_square25.jpg
[[0.29431987 0.7056802 ]] [1] {'focused': 0, 'unfocused': 1}
test_blur/black_square33.jpg
[[0.2458293  0.75417066]] [1] {'focused': 0, 'unfocused': 1}
test_blur/black_square41.jpg
[[0.22120003 0.77879995]] [1] {'focused': 0, 'unfocused': 1}
